# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4873, uptime 0:00:33
mini_beamline                    RUNNING   pid 4874, uptime 0:00:33
random_walk                      RUNNING   pid 4875, uptime 0:00:33
random_walk_horiz                RUNNING   pid 4876, uptime 0:00:33
random_walk_vert                 RUNNING   pid 4877, uptime 0:00:33
simple                           RUNNING   pid 4878, uptime 0:00:33
thermo_sim                       RUNNING   pid 4879, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4880, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-20 15:24:05
Persistent Unique Scan ID: '88d9b5a8-bb75-4e5c-b8ab-ee43c9aa5284'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 15:24:05.8 |     -1.000 |      96930 |


|         2 | 15:24:05.8 |     -0.500 |     101218 |
|         3 | 15:24:05.8 |      0.000 |     101413 |
|         4 | 15:24:05.8 |      0.500 |     102211 |
|         5 | 15:24:05.9 |      1.000 |     100045 |


+-----------+------------+------------+------------+
generator scan ['88d9b5a8'] (scan num: 3)





('88d9b5a8-bb75-4e5c-b8ab-ee43c9aa5284',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/88d9b5a8-bb75-4e5c-b8ab-ee43c9aa5284-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-20 15:24:06
Persistent Unique Scan ID: 'b97a2682-ea48-4e7d-a730-afb11f74bfb9'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 15:24:06.3 |          -1 | 1122093712 |


|         2 | 15:24:06.4 |          -0 | 1125952126 |


|         3 | 15:24:06.5 |           0 | 1130579118 |


|         4 | 15:24:06.7 |           0 | 1131798426 |


|         5 | 15:24:06.8 |           1 | 1131534026 |
+-----------+------------+-------------+------------+
generator scan ['b97a2682'] (scan num: 4)





('b97a2682-ea48-4e7d-a730-afb11f74bfb9',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-20 15:24:07
Persistent Unique Scan ID: 'e09560e2-d7c2-45c8-b140-08a29efb3ec1'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 15:24:07.3 | 1121008675 |


|         2 | 15:24:08.2 | 1063423029 |


|         3 | 15:24:09.2 | 1066428748 |


...

|        28 | 15:24:34.2 | 1123437367 |


|        29 | 15:24:35.2 | 1116618489 |


|        30 | 15:24:36.2 | 1061310741 |


+-----------+------------+------------+
generator count ['e09560e2'] (scan num: 5)





('e09560e2-d7c2-45c8-b140-08a29efb3ec1',)